In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd

In [3]:
idx2char=['h','i','e','l','o']
#hihello 문자열, hihell -> hihell(o)

In [4]:
xdata=[[0,1,0,2,3,3]] #<-hihell 까지 입력
#[None, sequence_length,input_dim]: xdata의 shape

In [5]:
xOneHot=[[
    [1,0,0,0,0], #<- h를 의미
    [0,1,0,0,0], #<- i를 의미
    [1,0,0,0,0], #<- h를 의미
    [0,0,1,0,0], #<- e를 의미
    [0,0,0,1,0], #<- l를 의미
    [0,0,0,1,0], #<- l를 의미
]]
#hihell이 입력되면 ihello가 출력되어야함
ydata=[[1,0,2,3,3,4]]
np.shape(xOneHot) #<- (문장1개,6글자,원핫인코딩시 크기5)

(1, 6, 5)

In [6]:
num_classes = 5 #<- class가 5개
input_dim = 5     #<- 원핫 크기
hidden_size = 5    #<- RNN(LSTM) cell에서 출력 크기
batch_size = 1     #<- 문장이 1개이니까
sequence_length = 6   #<- 출력되는 문자의 수
learning_rate = 0.1

In [7]:
x = tf.placeholder(tf.float32,                         #원핫의 크기: 중복되지 않은 글자의 수
                  [None, sequence_length,input_dim])  #<- 6글자가 들어감,원핫의 크기5
y = tf.placeholder(tf.int32,   #<- 나중에 연산하면 error나와서 일단 int로
                  [None, sequence_length])

In [8]:
# cell로 정의한 것이 cell state임
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size,
                                    state_is_tuple=True) #<-상태 정보를 tuple로?
initial_state = cell.zero_state(batch_size,tf.float32) #<- 0으로 초기화 하겠냐?
output, _states = tf.nn.dynamic_rnn(
cell,x,initial_state=initial_state,dtype=tf.float32
) 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
xForFc = tf.reshape(output,[-1,hidden_size])
output = tf.contrib.layers.fully_connected(
    inputs=xForFc, num_outputs=num_classes,
    activation_fn=None
)
output = tf.reshape(output,
                    [batch_size,
                            sequence_length,
                            num_classes])

In [13]:
weights = tf.ones([batch_size,sequence_length])

In [15]:
sequence_loss = tf.contrib.seq2seq.sequence_loss(
logits=output, targets=y, weights=weights
)

In [17]:
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [19]:
prediction = tf.argmax(output, axis=2)

In [35]:
tf.reset_default_graph()

In [36]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        lossv,_ =sess.run([loss,train],feed_dict={x:xOneHot, y:ydata})
        res=sess.run(prediction,feed_dict={x:xOneHot})
        print(i,"cost:",lossv,
              "prediction:",res,
              "true Y:",ydata)
        s=[idx2char[c] for c in np.squeeze(res)]
        print("\n prediction str:","".join(s))

TypeError: unhashable type: 'numpy.ndarray'

In [27]:
x=np.array([1,2])
print(x.shape)
y=np.expand_dims(x,axis=0)
y.shape #<- 대괄호가 하나 더 생김(차원증가)

(2,)


(1, 2)

In [26]:
y #<-앞쪽에 차원이 추가됨

array([[1, 2]])

In [29]:
z = np.expand_dims(x,axis=1)
print(z,z.shape)

[[1]
 [2]] (2, 1)


In [32]:
x=np.array([[[0],[1],[2]]])
print(x.shape,x)

(1, 3, 1) [[[0]
  [1]
  [2]]]


In [31]:
np.squeeze(x)

array([0, 1, 2])

In [45]:
tf.reset_default_graph()

In [46]:
hidden_size=2
cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)

In [47]:
xdata=np.array([[[1,0,0,0]]],dtype=np.float32)
#xdata shape=(1,1,4)
#출력 shape(1,1,4)

In [49]:
sess=tf.Session()
outputs,_states = tf.nn.dynamic_rnn(cell,xdata,dtype=tf.float32)
sess.run(tf.global_variables_initializer())
print(sess.run(outputs))   #hidden_size:2로 잡아서 input:(1,1,4)=>output:(1,1,2)

[[[ 0.04116742 -0.0565173 ]]]


In [ ]:
"""
입력 shape 계산 방법
ex: (3,5,4)인 경우
[[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],
[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],
[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]]

input shape(단어갯수,글자수,원핫크기)
밖에서 안으로 들어가면서 숫자를 세면 됨
"""

In [ ]:
tf.reset_default_graph()

In [ ]:
# h -> [1,0,0,0], e
xdata=np.array([[[h,e,l,l,o],
                [e,o,l,l,l], #<-(3,5)로 보이지만 알파벳당 원핫인코딩 하면 (3,5,4)가 됨
                [l,l,e,e,l]]],dtype=np.float32)

In [50]:
hidden_size=2 #<- 출력이 2개란 의미
cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size) 
sess=tf.Session()
outputs,_states = tf.nn.dynamic_rnn(cell,xdata,dtype=tf.float32)#<- RNN을 실제로 구동함
sess.run(tf.global_variables_initializer())
print(sess.run(outputs))      

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)


In [ ]:
"""
mnist => RNN 가능?
위에서 아래로 내려가는 경향이 있으니 위부터 아래로 내려가는 시간으로 생각하고 할 수 있음
3D로 차원을 늘려서 해도 되는데 계산이 엄청나게 복잡해짐

"""

In [51]:
from tensorflow.examples.tutorials.mnist import input_data

In [52]:
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)
lr = 0.01
total_epoch = 30 
batch_size = 128

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [53]:
n_input = 28 #<- 한번에 입력받는 데이터 개수
n_step = 28 #<- 28줄
n_hidden = 128 #<- cell에서 나가는 출력의 개수
n_class = 10 #<- 숫자의 종류(0~9)

In [61]:
w = tf.Variable(tf.random_normal([n_hidden,n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [54]:
x = tf.placeholder(tf.float32, [None,n_step,
                               n_input])
y = tf.placeholder(tf.float32, [None,n_class])

In [69]:
#RNN 학습할 셀 생성
cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden) #<- cell

In [72]:
#RNN 망 구성, cell을 먼저 만들어놔야 RNN망을 만들 수 있음
outputs,state = tf.nn.dynamic_rnn(cell,x,dtype=tf.float32)
#outputs: [batch_size,n_step,n_hidden ]
# => 구조 변경(y의형태로) => y:[batch_szie,n_class]

#     0         1       2 
#[batch_size,n_step,n_hidden] - 변환 ->
#[n_step,batch_size,n_hidden]
#     1        0        2    
outputs = tf.transpose(outputs,[1,0,2])
outputs = outputs[-1]

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)


In [71]:
model = tf.matmul(outputs,w)+b

ValueError: Dimensions must be equal, but are 2 and 128 for 'MatMul_1' (op: 'BatchMatMulV2') with input shapes: [1,1,2], [128,10].

In [65]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
                                                                labels=y))
opt = tf.train.AdamOptimizer(lr).minimize(cost)

NameError: name 'model' is not defined

In [66]:
#신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [67]:
total_batch = int(mnist.train.num_examples/batch_size)

In [68]:
for epoch in range(total_epoch):
    total_cost=0
    for i in range(total_batch):
        batchX,batchY = mnist.train.next_batch(batch_size)
        batchX = batchX.reshape((batch_size,n_step,n_input))
        #128,784 -> 128,28,28로 reshape
        _,costv = sess.run([opt,cost],feed_dict={x:batchX,y:batchY})
        total_cost+=costv
    print("epoch:","%4d" % (epoch+1),
         "cost average:","{:.3f}".format(total_cost/total_batch))
    

NameError: name 'opt' is not defined

In [74]:
isCorrect = tf.equal(tf.argmax(model,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(isCorrect,tf.float32))
testSize = len(mnist.test.images)
testX = mnist.test.images.reshape(testSize,n_step,n_input)
testY = mnist.test.labels

print(sess.run(accuracy,feed_dict={x:testX,y:testY}))

NameError: name 'model' is not defined